In [ ]:
!pip install -U d2l

In [ ]:
!pip install skorch

In [1]:
%matplotlib inline
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn
from torchsummary import summary
from d2l import torch as d2l
import numpy as np
from tensorflow import keras
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from skorch import NeuralNetClassifier

In [2]:
def initialize_model(model_name, num_classes=10, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = torchvision.models.resnet18(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        nn.init.xavier_uniform_(model_ft.fc.weight);

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft

# KNN on original input

In [ ]:
#ref : https://towardsdatascience.com/multiclass-classification-using-k-nearest-neighbours-ca5281a9ef76

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

knn = KNeighborsClassifier()

#convert to 2d
x_train = np.reshape(x_train,(x_train.shape[0],-1))
x_test = np.reshape(x_test,(x_test.shape[0],-1))

knn.fit(x_train, y_train.ravel())

ypred = knn.predict(x_test)

acc = accuracy_score(y_test, ypred)

print(acc)

# KNN on last layer

In [3]:
#softmax
#ref : implementation from tutorial 7 answer by Harrison
# https://www.youtube.com/watch?v=NVcJhMtRErE&list=PLyd8fQKbk5ej_lNdyAKI5aS_4hm-FhrrF&index=8
class NewModel(nn.Module):
    def __init__(self, pretrained_model, num_layers):
        super(NewModel, self).__init__()
        self.pretrained = pretrained_model
        self.flatten = nn.Flatten()
        #self.output = output_model
        #ref : https://forums.fast.ai/t/pytorch-best-way-to-get-at-intermediate-layers-in-vgg-and-resnet/5707
        self.features = nn.Sequential(*list(self.pretrained.children())[:num_layers])

    def forward(self, x):
        return self.flatten.forward(self.features(x))
        #return self.forward(out)
        #return out

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train.astype(np.float32)

x_train, y_train, x_test, y_test = map(
    torch.tensor, (x_train, y_train, x_test, y_test)
)

resnet = initialize_model('resnet')

x_train = x_train.permute(0, 3, 1, 2)

model = NewModel(resnet, 8)
out = model(x_train)
#print(out.shape)
print(y_train.type)
knn = KNeighborsClassifier(n_neighbors=5,algorithm='kd_tree')
knn.fit(out, y_train)

#knn = KNeighborsClassifier

#knn.fit(out, y_train)

#x_train /= 255.0

#x_train = x_train.permute(0, 3, 1, 2)

#x_train /= 255.0

#x_train.shape

#model(x_train)
